In [ ]:
!apt-get install -y --no-install-recommends

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
%%writefile bigger3.cu
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>
#include <cuda_runtime_api.h>
#include <cuda_device_runtime_api.h>
#include <device_launch_parameters.h>
#include <device_functions.h>
#include <cooperative_groups.h>
#include <ctime>


using namespace std;


#define width 8192
#define NUMS 1024
#define width2 (width * width) // 67,108,864
#define pixelBlockSize 16
#define totalPixelBlocks (width2 / (pixelBlockSize * pixelBlockSize)) // 262,144
#define totalThreads totalPixelBlocks // 262,144
#define threadPerBlock 8
#define GRID (totalThreads / threadPerBlock) // 32,768



#define cudaCheckErrors(msg)                         \
    do                                               \
    {                                                \
        cudaError_t __err = cudaGetLastError();      \
        if (__err != cudaSuccess)                    \
        {                                            \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n", \
                    msg, cudaGetErrorString(__err), \
                    __FILE__, __LINE__);            \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1);                                 \
        }                                            \
    } while (0)
/////////////////////////////////////////////////////////////////////////////
__global__ void first_scenario(int* d_results, short* d_matrix, long size)
{
    int index = threadIdx.x;
    d_results[index] = 0;

    for (int i = 0; i < size; i++)
    {
        int element = d_matrix[i];
        if (element == index)
        {
            d_results[index] = d_results[index] + 1;
        }
    }
}


/////////////////////////////////////////////////////////////////////////////
__global__ void second_scenario(int* d_results2, int* d_mid, short* d_matrix)
{
    int tidx = threadIdx.x;

    __shared__ int partialCount[threadPerBlock][NUMS];

    for (int i = 0; i < NUMS; i++)
    {
        partialCount[tidx][i] = 0;
    }

    __syncthreads();

    int startIndex = (blockIdx.x * (width2 / GRID)) + (tidx * (pixelBlockSize * pixelBlockSize));
    if (startIndex < width2)
    {
        for (int i = 0; i < (pixelBlockSize * pixelBlockSize); i++)
        {
            int globalIdx = startIndex + i;
            if (globalIdx < width2)
            {
                int value = d_matrix[globalIdx];
                partialCount[tidx][value]++;
            }
            else
                printf("globalIdx is out of bound\n");
        }
    }
    else
        printf("startIndex is out of bound\n");

    __syncthreads();

    if (tidx == 0)
    {
        for (int n = 0; n < NUMS; n++)
        {
            for (int t = 0; t < threadPerBlock; t++)
            {
                int midIdx = blockIdx.x * NUMS + n;
                atomicAdd(&d_mid[midIdx], partialCount[t][n]);
            }
        }
    }

    __syncthreads();

    if (tidx == 2 && blockIdx.x == 0)
    {
        for (int i = 0; i < NUMS; i++)
        {
            for (int j = 0; j < GRID; j++)
            {
                int resIndex = i + j * NUMS;
                atomicAdd(&d_results2[i], d_mid[resIndex]);
            }
        }
    }
}



/////////////////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////////////////


int main()
{
    short* d_matrix;
    int* d_results;


    cout << "CPU Matrix Allocated!" << endl;

    // Allocate and initialize matrix on the host
    short* h_matrix = new short[width2];
    for (int i = 0; i < width2; i++)
    {
        h_matrix[i] = rand() % NUMS;
    }

    // Allocate memory on the device for the matrix
    cudaMalloc((void**)&d_matrix, width2 * sizeof(short));
    cudaCheckErrors("cudaMalloc problem");

    // Copy the matrix from host to device
    cudaMemcpy(d_matrix, h_matrix, width2 * sizeof(short), cudaMemcpyHostToDevice);
    cudaCheckErrors("cudaMemcpy problem");

    cout << "GPU Matrix Allocated!" << endl;
    cout << endl << "Random initialization is Done!" << endl;
    cout << "Data transfer Successful." << endl;

    // Allocate memory on the device for results
    cudaMalloc((void**)&d_results, NUMS * sizeof(int));
    cudaCheckErrors("cudaMalloc problem");

    // Launch the first scenario kernel
    clock_t start1 = clock();
    first_scenario<<<1, NUMS>>>(d_results, d_matrix, width2);
    cudaCheckErrors("kernel_launch problem");
    cudaDeviceSynchronize();
    clock_t end1 = clock();
    cudaCheckErrors("synchronization");

    int* d_results2;
    cudaMalloc((void**)&d_results2, NUMS * sizeof(int));
    cudaCheckErrors("cudaMalloc problem");

    int* d_mid;
    int midSize = GRID * NUMS;
    cudaMalloc((void**)&d_mid, midSize * sizeof(int));
    cudaCheckErrors("cudaMalloc problem");

    // Zero-initialize the intermediate results on the device
    cudaMemset(d_mid, 0, midSize * sizeof(int));
    cudaCheckErrors("cudaMemset problem");

    cout << "Second kernel starts..." << endl;

    // Launch the second scenario kernel
    clock_t start = clock();
    second_scenario<<<GRID, threadPerBlock>>>(d_results2, d_mid, d_matrix);
    cudaCheckErrors("kernel_launch problem");

    cudaDeviceSynchronize();
    clock_t end = clock();
    cudaCheckErrors("synchronization");

    cout << "Second scenario ended" << endl;

    // Copy results from device to host
    int* h_results = new int[NUMS];
    cudaMemcpy(h_results, d_results, NUMS * sizeof(int), cudaMemcpyDeviceToHost);
    cudaCheckErrors("cudaMemcpy problem");

    int* h_results2 = new int[NUMS];
    cudaMemcpy(h_results2, d_results2, NUMS * sizeof(int), cudaMemcpyDeviceToHost);
    cudaCheckErrors("cudaMemcpy problem");

    // Print the results
    for (int i = 0; i < NUMS; i++)
    {
        cout << i << "-" << h_results[i] << "-" << h_results2[i] << endl;
    }

  float seconds1 = (float)(end1 - start1) / CLOCKS_PER_SEC;
	float seconds = (float)(end - start) / CLOCKS_PER_SEC;

	cout << "First Scenario ended in: " << seconds1 << endl;
	cout << "Second Scenario ended in: " << seconds << endl;

    // Free allocated memory on the device
    cudaFree(d_matrix);
    cudaFree(d_results);
    cudaFree(d_results2);
    cudaFree(d_mid);

    // Free allocated memory on the host
    delete[] h_matrix;
    delete[] h_results;
    delete[] h_results2;

    return 0;
}


Overwriting bigger3.cu


In [ ]:
!nvcc bigger3.cu -o cuda_code_0

In file included from bigger3.cu:7:
/usr/local/cuda/bin/../targets/x86_64-linux/include/device_functions.h:54:2: warning: #warning "device_functions.h is an internal header file and must not be used directly.  This file will be removed in a future CUDA release.  Please use cuda_runtime_api.h or cuda_runtime.h instead." [-Wcpp]
   54 | #warning "device_functions.h is an internal header file and must not be used directly.  This file will be removed in a future CUDA release.  Please use cuda_runtime_api.h or cuda_runtime.h instead."
      |  ^~~~~~~
In file included from bigger3.cu:7:
/usr/local/cuda/bin/../targets/x86_64-linux/include/device_functions.h:54:2: warning: #warning "device_functions.h is an internal header file and must not be used directly.  This file will be removed in a future CUDA release.  Please use cuda_runtime_api.h or cuda_runtime.h instead." [-Wcpp]
   54 | #warning "device_functions.h is an internal header file and must not be used directly.  This file will be remo

In [ ]:
!./cuda_code_0

CPU Matrix Allocated!
GPU Matrix Allocated!

Random initialization is Done!
Data transfer Successful.
Second kernel starts...
Second scenario ended
0-65307-257
1-65396-35394
2-65758-65758
3-65596-65596
4-65768-65768
5-65299-65299
6-65305-65305
7-65575-65575
8-65389-65389
9-65640-65640
10-65266-65266
11-65374-65374
12-65013-65013
13-65482-65482
14-65215-65215
15-65463-65463
16-64758-64758
17-65351-65351
18-65180-65180
19-65368-65368
20-65355-65355
21-65390-65390
22-65028-65028
23-65542-65542
24-65436-65436
25-65584-65584
26-65535-65535
27-65759-65759
28-65333-65333
29-65856-65856
30-65551-65551
31-65577-65577
32-65920-65920
33-65567-65567
34-65301-65301
35-65828-65828
36-65432-65432
37-65256-65256
38-65661-65661
39-65379-65379
40-65605-65605
41-65517-65517
42-65339-65339
43-65424-65424
44-65376-65376
45-65361-65361
46-65776-65776
47-65344-65344
48-65485-65485
49-65308-65308
50-65147-65147
51-65713-65713
52-65377-65377
53-66135-66135
54-64984-64984
55-65329-65329
56-65573-65573
57-65378-

In [ ]:
!cuda-memcheck ./cuda_code_0

========= CUDA-MEMCHECK
========= This tool is deprecated and will be removed in a future release of the CUDA toolkit
========= Please use the compute-sanitizer tool as a drop-in replacement
========= Error: Could not find ./cuda_code_0
========= ERROR SUMMARY: 0 errors
